<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Some-notes" data-toc-modified-id="Some-notes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Some notes</a></span><ul class="toc-item"><li><span><a href="#Cost-functions" data-toc-modified-id="Cost-functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Cost functions</a></span></li></ul></li><li><span><a href="#Load" data-toc-modified-id="Load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Viz-costs" data-toc-modified-id="Viz-costs-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Viz costs</a></span></li><li><span><a href="#Solving-just-one-instance-to-check-if-it-works-properly" data-toc-modified-id="Solving-just-one-instance-to-check-if-it-works-properly-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Solving just one instance to check if it works properly</a></span></li><li><span><a href="#Sampling-solutions" data-toc-modified-id="Sampling-solutions-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Sampling solutions</a></span></li><li><span><a href="#Run-complete-algorithm" data-toc-modified-id="Run-complete-algorithm-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Run complete algorithm</a></span></li></ul></li></ul></div>

In [ ]:
from matplotlib import rc
rc('text', usetex=True)
rc('font', size = 12)
rc('xtick', labelsize = 12)
rc('ytick', labelsize = 12)
rc('figure', figsize = (8, 4))

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from amod_ed.contractivity_v3 import viz_costs, _construct_problem
from amod_ed.contractivity_v3 import sample_solutions, run_algorithm
import cvxpy as cp
import networkx as nx

import pandas as pd

from amod_ed.contractivity_v3 import plot_results_run

from amod_ed.contractivity_v3 import get_new_r, get_edge_flow

# Description

This notebook implements a more general framework, to be essentially tested on a 3-node case. 

TODO: 
- Check for failures
- Test with different scenarios (i.e. parameters of the network)

# Some notes

## Cost functions

In the different functions of the code, the cost functions and inverse demand functions are defined based on BPR functions (out of simplicity). 

We take the following notation: 

$$
BPR(x,\kappa, \phi) = \phi(1+\alpha (x/\kappa)^\beta),
$$
where $\alpha$ and $\beta$ are parameters. 

The edge costs are actually BPR functions. As for the inverse demand functions, we define them as
$$
D^{-1} = - BPR() + S,
$$
where $S\in\mathbb{R}^+$ is the inverse demand shift, ensuring that $D^{-1}$ is larger than zero over a given subset of $\mathbb{R}^+$.

# Load

The characteristics of the graph are loaded from Excel files. 

In [ ]:
edges = pd.read_excel('cost_edges_3.xlsx')
inv_edges = pd.read_excel('inv_demand_3.xlsx')

In [ ]:
edges

In [ ]:
inv_edges

## Viz costs

Visualize the costs for each OD pair specified in the Excels. 

In [ ]:
viz_costs(edges, inv_edges)

## Solving just one instance to check if it works properly

The below builds the optimization problem. 

In [ ]:
"""
Returns
-------
f_p: cvxpy.Variable
    The flow for each commodity on each edge
f_r: cvxpy.Variable
    The rebalancing flow on each edge
r: cvxpy.Parameter
    The rebalancing guess for each node
d_var: cvxpy.Variable
    The demand for each each
prob: cvxpy.Problem
    The optimization problem
map_comps: dict
    A map linking components of f_p to the edges and inv edges
map_edges: dict
    A map linking edges to components of f_p
costs_dict: dict
    Dict containing the cost for each edge
inv_demand_dict: dict
    The inverse demand cost for each od pair
G: nx.DiGraph
    Graph representing the network
nodes: list
    list of nodes
"""
f_p, f_r, r, d_var, prob, map_comps, map_edges,\
 costs_dict, inv_d_dict, G, nodes = _construct_problem(edges, inv_edges)

Specify a value of the rebalancing parameter

In [ ]:
r_new = [0, 0, 0]
r.value = r_new

You can then solve the problem. 

In [ ]:
prob.solve(solver = cp.GUROBI)

Check the status. 

In [ ]:
prob.status

You can retrieve the values of the passenger flow. However it is difficult to understand those larger dimensional vectors.

In [ ]:
f_p.value

Therefore, the below function helps in analyzing and decomposing edge by edge for the passenger flow. 

In [ ]:
get_edge_flow(f_p, map_edges)

Those dictionnaries enable to check the cost of each edge. 

We can therefore compare the cost of paths to the cost of the inverse demand. 

In [ ]:
costs_dict[(1,2)].value

In [ ]:
costs_dict[(1,3)].value + costs_dict[(3, 2)].value

In [ ]:
inv_d_dict[(1,2)].value

We can also compute directly the new rebalancing guess. 

In [ ]:
get_new_r(f_p, map_edges, nodes)

## Sampling solutions

We run a given number of times and check the ratio to probe contractivity. 
You can play with 
- seed
- nsamples

In [ ]:
Tr, r_, dT, dr = sample_solutions(edges, inv_edges, nsamples = 1000, seed=0)

## Run complete algorithm

You can also run the complete algorithm for a given number of iterations. 

In [ ]:
r_tot = run_algorithm(edges, inv_edges, nsolutions = 5, seed =0, max_iter = 15)

The below functions plots the following:
- One graph for the evolution of each component of r (3 graphs)
- One graph for the evolution of each component in the last 5 iterations (3 graphs)
- One graph for the evolution of the difference of the norm between two iterations (1 graph)

In [ ]:
plot_results_run(r_tot)